In [1]:
from utils import load_model_and_tokenizer

model, tokenizer = load_model_and_tokenizer()

In [1]:
from utils import generate_text

prompt = "generate a piece that is around 1 minute long and only piano from what you learned"

text = generate_text(prompt)

print(text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Saved generated text to ../example_outputs/model_output/output_7.txt
Generated_Symphony_373>>>1.00>Piano: Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_


In [ ]:
from prompt_utils import parse_user_instruction

prompt = "generate a piece named wow that is around 1 minute long and use piano and some other instruments using something like 190 bpm from what you learned"

out_dict = parse_user_instruction(prompt)

out_dict

{'name': 'wow',
 'bpm': 190,
 'strict_bpm': True,
 'duration_minutes': 1.0,
 'instruments': ['piano', 'trumpet', 'drums'],
 'raw_text': 'generate a piece named wow that is around 1 minute long and use piano and some other instruments using something like 190 bpm from what you learned'}

In [ ]:
from prompt_utils import build_structured_prompt

prompt2, meta = build_structured_prompt(tokenizer, out_dict)

prompt2, meta 

('<|startofpiece|><NAME=wow><BPM=190><DURATION_BEATS=190><DURATION_MINUTES=1.00><TRACKS>Piano: Rest_q Rest_q Rest_q <TRACKSEP> Trumpet: Rest_q Rest_q Rest_q <TRACKSEP> Drums: Rest_q Rest_q Rest_q',
 {'name': 'wow',
  'bpm': 190.0,
  'duration_beats': 190,
  'duration_minutes': 1.0,
  'instruments': ['piano', 'trumpet', 'drums']})

In [16]:
text = generate_text(prompt2)

text

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Saved generated text to ../example_outputs/model_output/output_6.txt


'wow>190>190>1.00>Piano: Rest_q Rest_q Rest_q  Trumpet: Rest_q Rest_q Rest_q  Drums: Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_q Rest_'